<a href="https://colab.research.google.com/github/varekarprajwal/DownLoader/blob/main/Archive_Manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#@markdown ▶️ **Run this cell to mount your Google Drive**
from google.colab import drive
import os

try:
    drive.mount('/content/drive', force_remount=True)
    print("\n✅ Google Drive mounted successfully!")
    # Create a symbolic link for easier access
    if not os.path.lexists('/mydrive'):
        os.symlink('/content/drive/MyDrive', '/mydrive')
    print("💡 You can now use '/mydrive/' as a shortcut for '/content/drive/MyDrive/'.")
except Exception as e:
    print(f"🚨 An error occurred while mounting Google Drive: {e}")

Mounted at /content/drive

✅ Google Drive mounted successfully!
💡 You can now use '/mydrive/' as a shortcut for '/content/drive/MyDrive/'.


In [14]:
#@markdown ### 📂 **Decompression Settings**
archive_path = "/content/drive/MyDrive/GoodNotes.zip" #@param {type:"string"}
destination_path = "/content/drive/MyDrive/GATE/" #@param {type:"string"}
password = "" #@param {type:"string"}

# =============================================
# IMPORTS
import os
import sys
import zipfile
import tarfile
import subprocess
from tqdm.notebook import tqdm
from google.colab import output

# =============================================
# LOGIC
def extract_archive(source, dest, pwd=None):
    """Extracts a supported archive file, clearing the output on completion."""
    final_message = ""
    with output.use_tags('decompress_output'):
      try:
          if not os.path.exists(source):
              raise FileNotFoundError(f"Source file not found at '{source}'")

          os.makedirs(dest, exist_ok=True)
          filename = os.path.basename(source)

          if source.endswith('.zip'):
              sys.stdout.write(f"📦 Extracting ZIP file: {filename}\n")
              sys.stdout.flush()
              with zipfile.ZipFile(source, 'r') as zip_ref:
                  if pwd:
                      zip_ref.setpassword(pwd.encode())
                  members = zip_ref.infolist()
                  for member in tqdm(members, desc="Extracting"):
                      zip_ref.extract(member, dest)

          elif source.endswith('.rar'):
              sys.stdout.write(f"📦 Extracting RAR file: {filename}\n")
              sys.stdout.flush()
              command = ['unrar', 'x']
              if pwd:
                  command.append(f'-p{pwd}')
              else:
                  command.append('-y')
              command.extend([source, dest])

              process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
              while True:
                  out = process.stdout.readline()
                  if out == '' and process.poll() is not None:
                      break
                  if out:
                      sys.stdout.write(out)
                      sys.stdout.flush()
              rc = process.poll()
              if rc != 0:
                _, stderr = process.communicate()
                raise Exception(f"Unrar Error: {stderr.strip()}")

          elif source.endswith(('.tar', '.tar.gz', '.tar.bz2')):
              sys.stdout.write(f"📦 Extracting TAR archive: {filename}\n")
              sys.stdout.flush()
              mode = 'r'
              if source.endswith('.tar.gz'): mode = 'r:gz'
              elif source.endswith('.tar.bz2'): mode = 'r:bz2'

              with tarfile.open(source, mode) as tar_ref:
                  members = tar_ref.getmembers()
                  for member in tqdm(members, desc="Extracting"):
                      tar_ref.extract(member, dest)
          else:
              raise ValueError(f"Unsupported file format for '{filename}'.")

          final_message = f"✅ Successfully extracted '{filename}' to '{dest}'"

      except Exception as e:
          final_message = f"❌ Error: {e}"

    output.clear(output_tags='decompress_output')
    print(final_message)


# =============================================
# EXECUTION
if not archive_path or not destination_path:
    print("⚠️ Please provide both the archive path and the destination path.")
else:
    extract_archive(archive_path, destination_path, password)

✅ Successfully extracted 'GoodNotes.zip' to '/content/drive/MyDrive/GATE/'


In [15]:
#@markdown ### 🗜️ **Compression Settings**
archive_type = "zip"  #@param ["zip", "rar", "tar", "tar.gz", "tar.bz2"]
source_path = "/content/drive/MyDrive/GoodNotes"  #@param {type:"string"}
destination_path = "/content/drive/MyDrive"  #@param {type:"string"}
password = ""  #@param {type:"string"}

# =============================================
# IMPORTS
import os
import sys
import zipfile
import tarfile
import subprocess
from tqdm.notebook import tqdm
from google.colab import output

# =============================================
# LOGIC
def compress_path(source, dest, archive_format, pwd=None):
    """Compresses a file or folder, clearing the output on completion."""
    final_message = ""
    with output.use_tags('compress_output'):
      try:
          if not os.path.exists(source):
              raise FileNotFoundError(f"Source path not found at '{source}'")

          dest_dir = os.path.dirname(dest)
          if dest_dir:
              os.makedirs(dest_dir, exist_ok=True)

          filename = os.path.basename(dest)
          sys.stdout.write(f"⚙️ Starting compression to '{filename}'...\n")
          sys.stdout.flush()

          if archive_format == 'zip':
              with zipfile.ZipFile(dest, 'w', zipfile.ZIP_DEFLATED) as zipf:
                  if pwd:
                      zipf.setpassword(pwd.encode())

                  if os.path.isdir(source):
                      file_list = [os.path.join(r, f) for r, d, files in os.walk(source) for f in files]
                      with tqdm(total=len(file_list), desc="Compressing") as pbar:
                          for file_path in file_list:
                              arcname = os.path.relpath(file_path, os.path.dirname(source))
                              zipf.write(file_path, arcname=arcname)
                              pbar.update(1)
                  else:
                      zipf.write(source, os.path.basename(source))

          elif archive_format == 'rar':
              sys.stdout.write("💡 Installing 'rar' command-line tool...\n")
              sys.stdout.flush()
              install_proc = subprocess.run(['apt-get', 'install', '-y', 'rar'], capture_output=True, text=True, check=False)
              if install_proc.returncode != 0:
                  raise Exception(f"Failed to install rar: {install_proc.stderr}")

              command = ['rar', 'a']
              if pwd:
                  command.append(f'-hp{pwd}')
              command.extend(['-r', dest, source])

              sys.stdout.write("📦 Creating RAR archive...\n")
              sys.stdout.flush()
              subprocess.run(command, check=True)

          elif archive_format in ['tar', 'tar.gz', 'tar.bz2']:
              mode = 'w'
              if archive_format == 'tar.gz': mode = 'w:gz'
              elif archive_format == 'tar.bz2': mode = 'w:bz2'

              with tarfile.open(dest, mode) as tarf:
                  tarf.add(source, arcname=os.path.basename(source))

          final_message = f"✅ Successfully created archive '{filename}'"

      except Exception as e:
          final_message = f"❌ Error: {e}"

    output.clear(output_tags='compress_output')
    print(final_message)

# =============================================
# EXECUTION
if not source_path or not destination_path:
    print("⚠️ Please provide both the source path and the destination path.")
else:
    extension_map = {
        'zip': '.zip', 'rar': '.rar', 'tar': '.tar',
        'tar.gz': '.tar.gz', 'tar.bz2': '.tar.bz2'
    }
    expected_extension = extension_map[archive_type]
    final_dest_path = destination_path.strip()

    if os.path.isdir(final_dest_path) or final_dest_path.endswith('/'):
        source_basename = os.path.basename(source_path.rstrip('/\\'))
        final_dest_path = os.path.join(final_dest_path, source_basename + expected_extension)
        print(f"ℹ️ Destination is a directory. Auto-creating archive name.")
    elif not final_dest_path.endswith(expected_extension):
        final_dest_path += expected_extension
        print(f"ℹ️ Auto-added extension to destination path.")

    print(f"➡️ Final destination: {final_dest_path}")
    compress_path(source_path, final_dest_path, archive_type, password)

ℹ️ Destination is a directory. Auto-creating archive name.
➡️ Final destination: /content/drive/MyDrive/GoodNotes.zip
✅ Successfully created archive 'GoodNotes.zip'
